# Dehli-Indian-Housing-Analysis-Part-5-Predict-Rental-Price-with-Everything-Size-numBedroom-numBathroom-Location-Name-and-House-Type by Dumisani Maxwell Mukuchura

## Dataset Source: https://www.kaggle.com/datasets/bhavyadhingra00020/india-rental-house-price

### Contact: dumisanimukuchura@gmail.com | LinkedIn: https://www.linkedin.com/in/dumisani-maxwell-mukuchura-4859b7170/

#### This Project will explore the Dehli Indian Housing and build a model predicting the Rental Price from Everything i.e the following features: House Size, Number of Bedrooms, Number of Bathrooms, Location Name and House Type. A total of 5 features.